## Relax Data Science Challenge

### Cleaning and EDA 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [49]:
df1=pd.read_csv('takehome_users.csv', engine='python')

In [50]:
df1.columns=['user_id', 'creation_time', 'name', 'email', 'creation_source',
       'last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id']

In [51]:
#I think lets drop the name and email and invited_by_user_id because it complicates things unnecissarily
df1=df1.drop(columns=['name','email','invited_by_user_id'])
#what can we do with nan in last_session_creation_time other than drop nulls?
df1=df1.dropna()
#reset index to object id
df1=df1.set_index('user_id')

and now the long drawn out round about way to figure out whether someone is an adopted user or not.  Im sure there is a more concise way, but I am still learning.

In [52]:
df2=pd.read_csv('takehome_user_engagement.csv',engine='python')

In [53]:
test=df2.set_index(['user_id','time_stamp'])

In [54]:
user_list_possible=[]
user_list_negative=[]
for user in df2.user_id.unique():
    if len(test.loc[user]) >= 3: 
        user_list_possible.append(user)
    else:
        user_list_negative.append(user)

In [55]:
negative_df_list=[]
for user in user_list_negative:
    x=test.loc[user]
    x['adopted_user']=False
    x['user_id2']=user
    negative_df_list.append(x)

In [56]:
positive_df_list=[]
for user in user_list_possible:
    x=test.loc[user].reset_index()
    x.time_stamp=pd.to_datetime(x.time_stamp)
    x['days']=(x.time_stamp.diff())/np.timedelta64(1,'D')
    x=x.loc[x['days']>=1].set_index('time_stamp').resample('W').sum()
    x=x[['days']]
    if len(x.loc[x.days>=3])>=1:
        x['adopted_user']=True
        x['user_id2']=user
        positive_df_list.append(x)
    else:
        x['adopted_user']=False
        x['user_id2']=user
        negative_df_list.append(x)
positive_df_list_again=[]
for df in positive_df_list:
    x=df.iloc[[0]]
    positive_df_list_again.append(x)

In [57]:
x=pd.concat(positive_df_list_again)
y=pd.concat(negative_df_list)
df2=pd.concat([x,y])

In [58]:
df2=df2.groupby('user_id2').sum()
df2['adopted']=df2['adopted_user']>0
df2=df2[['adopted']]
df2

,adopted
user_id2,
1,False
2,True
3,False
4,False
5,False
...,...
11996,False
11997,False
11998,False


In [59]:
df=df1.join(df2,how='inner')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8823 entries, 1 to 12000
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               8823 non-null   object 
 1   creation_source             8823 non-null   object 
 2   last_session_creation_time  8823 non-null   float64
 3   opted_in_to_mailing_list    8823 non-null   int64  
 4   enabled_for_marketing_drip  8823 non-null   int64  
 5   org_id                      8823 non-null   int64  
 6   adopted                     8823 non-null   bool   
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 811.1+ KB


In [61]:
#lets change the last_session_creation_time to regular datetime
from datetime import datetime
test=df['last_session_creation_time'].apply(lambda x : datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
df['last_session_creation_time']=test
df['last_session_creation_time']=pd.to_datetime(df['last_session_creation_time'])
df['creation_time']=pd.to_datetime(df['creation_time'])

In [62]:
#and create a new variable called time_as_user which measures the time in days between account creation time
#and the last session creation time
df['time_as_user']=(df['last_session_creation_time']-df['creation_time'])/np.timedelta64(1,'D')
df=df.reset_index()
df=df.drop(columns=['last_session_creation_time','creation_time','user_id'])

In [64]:
df=pd.get_dummies(df)

In [66]:
df.columns

Index(['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id',
       'adopted', 'time_as_user', 'creation_source_GUEST_INVITE',
       'creation_source_ORG_INVITE', 'creation_source_PERSONAL_PROJECTS',
       'creation_source_SIGNUP', 'creation_source_SIGNUP_GOOGLE_AUTH'],
      dtype='object')

## Modeling

In [70]:
from sklearn.model_selection import train_test_split
X=df.drop(columns='adopted')
y=df['adopted']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.8,random_state=42)

In [72]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(criterion='gini')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.969542428105964

In [73]:
clf=RandomForestClassifier(criterion='entropy')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.969825754356141

Out of the box random forest works pretty good, and using the entropy criterion works best.  Im sure there is a better model to be used, but I have already spent 2.5 hours on this and I must move on.  With this Random Forest model I can predict whether the customer is an adopted user with roughly 97% accuracy.     